In [6]:
import xarray as xr

ds = xr.open_dataset("E/E_2003_GLEAM_v4.2b_YR.nc")
print(ds)

E = ds["E"]

# 空间平均（最基础版，不加面积权重）
E_global = E.mean(dim=["lat", "lon"])

# 看值
print(E_global.values)

<xarray.Dataset> Size: 26MB
Dimensions:  (time: 1, lat: 1800, lon: 3600)
Coordinates:
  * time     (time) datetime64[ns] 8B 2003-12-31
  * lat      (lat) float64 14kB 89.95 89.85 89.75 89.65 ... -89.75 -89.85 -89.95
  * lon      (lon) float64 29kB -179.9 -179.8 -179.8 ... 179.7 179.8 179.9
Data variables:
    E        (time, lat, lon) float32 26MB ...
[364.707]
